In [21]:
class Txn:
    def __init__(self, name, read_set=set(), write_set=set(), should_abort=False):
        self.name = name
        self.read_set = read_set
        self.write_set = write_set
        self.should_abort = should_abort
    def __str__(self):
        return self.name
    def __repr__(self):
        return self.name

In [22]:
# Sample data
# Lock table:
# X: [A, E]
# Y: [B]
# Z: [C, D, E]
# our adjacency list should look like this:
# A: [E]
# B: []
# C: [D, E]
# D: [E]

In [26]:
txns = [
    Txn("A", {1, 2}, {3, 4}),
    Txn("B", {3}, {5}),
    Txn("C", {4,5}, {1,2}),
    Txn("D", {1}, {2,5}),
    Txn("E", {4,5}, {1,3}),
]

In [33]:
from collections import defaultdict, deque

# 1) build the lock table
status = dict()
lock_table = defaultdict(set)  # maps record -> set (should be single element for E, multiple for S)

adj = { txn: set() for txn in txns }
indegrees = {v: 0 for v in adj}

for txn in txns:
    for key in txn.read_set:
        if key in status:
            if status[key] == "S":   # shared, no conflict
                lock_table[key].add(txn)
            else:                    # conflict
                for conflicting_txn in lock_table[key]:
                    if txn not in adj[conflicting_txn]:
                        adj[conflicting_txn].add(txn)
                        indegrees[txn] += 1
                lock_table[key] = set([txn])
                status[key] = "S"
        else:
            lock_table[key] = set([txn])
            status[key] = "S"
        
    for key in txn.write_set:
        for conflicting_txn in lock_table[key]:
            if txn not in adj[conflicting_txn]:
                adj[conflicting_txn].add(txn)
                indegrees[txn] += 1
        lock_table[key] = set([txn])
        status[key] = "E"

# lock_table
print(adj)
print(indegrees)

{A: {B, C}, B: {C, E}, C: {D}, D: {E}, E: set()}
{A: 0, B: 1, C: 2, D: 1, E: 2}


In [31]:
# Kahn's algorithm

indegrees = {v: 0 for v in adj}

# O(V + E)
for txn in adj:
    for nei in adj[txn]:
        indegrees[nei] += 1

indegrees

{A: 0, B: 1, C: 2, D: 1, E: 2}

In [15]:
q = deque([])
for txn in indegrees:
    if indegrees[txn] == 0:
        q.append(txn)
q

deque([A])

In [16]:
# say we complete A
txn = q.popleft()
for nei in adj[txn]:
    indegrees[nei] -= 1
    if indegrees[nei] == 0:
        q.append(nei)

q

deque([B])

In [17]:
# say we complete B
txn = q.popleft()
for nei in adj[txn]:
    indegrees[nei] -= 1
    if indegrees[nei] == 0:
        q.append(nei)

q

deque([C])

In [18]:
# say we complete C
txn = q.popleft()
for nei in adj[txn]:
    indegrees[nei] -= 1
    if indegrees[nei] == 0:
        q.append(nei)

q

deque([D])

In [19]:
# say we complete D
txn = q.popleft()
for nei in adj[txn]:
    indegrees[nei] -= 1
    if indegrees[nei] == 0:
        q.append(nei)

q

deque([])